**D22033** **Gujjula Poojitha**

**Exp-6:** Generate a random sample of 200 frm cars data. Fit fully grown D.T model, Pruned D.T model with Opt cost complexity parameter and a random forest model with 50 D.T's and with the default settings. Repeat the experiment 100 times & compare the variance using specific test observation.

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv(r"C:\Users\pooji\Downloads\cars.csv")
df.head()

,Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin
0,Chevrolet Chevelle Malibu,18.0,8,307.0,130,3504,12.0,70,US
1,Buick Skylark 320,15.0,8,350.0,165,3693,11.5,70,US
2,Plymouth Satellite,18.0,8,318.0,150,3436,11.0,70,US
3,AMC Rebel SST,16.0,8,304.0,150,3433,12.0,70,US
4,Ford Torino,17.0,8,302.0,140,3449,10.5,70,US


In [14]:
df = pd.get_dummies(df)
df

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Car_AMC Ambassador Brougham,Car_AMC Ambassador DPL,Car_AMC Ambassador SST,...,Car_Volkswagen Type 3,Car_Volvo 144ea,Car_Volvo 145e (sw),Car_Volvo 244DL,Car_Volvo 245,Car_Volvo 264gl,Car_Volvo Diesel,Origin_Europe,Origin_Japan,Origin_US
0,18.0,8,307.0,130,3504,12.0,70,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,15.0,8,350.0,165,3693,11.5,70,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,18.0,8,318.0,150,3436,11.0,70,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,16.0,8,304.0,150,3433,12.0,70,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,17.0,8,302.0,140,3449,10.5,70,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,27.0,4,140.0,86,2790,15.6,82,0,0,0,...,0,0,0,0,0,0,0,0,0,1
402,44.0,4,97.0,52,2130,24.6,82,0,0,0,...,0,0,0,0,0,0,0,1,0,0
403,32.0,4,135.0,84,2295,11.6,82,0,0,0,...,0,0,0,0,0,0,0,0,0,1
404,28.0,4,120.0,79,2625,18.6,82,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
X = df.drop(["MPG","Origin","Car","Model"],axis=1)  
y = df[['MPG']] 

In [16]:
X_train = X.iloc[1:,:]
y_train = y.iloc[1:,:]

In [17]:
test_obs = X.iloc[0,:].values               
test_obs

array([   8.,  307.,  130., 3504.,   12.])

In [21]:
#Fitting Decision Tree with out pruning
DT_pred=[]
DT = DecisionTreeRegressor(random_state=0)
for i in range(100):
    X_sample = X_train.sample(200,random_state=i)   #sample size 200
    y_sample = y_train.sample(200,random_state=i) 
    DT.fit(X_sample,y_sample)                       #decision tree model
    DT_pred.append(DT.predict([test_obs])[0])

In [22]:
print("Variance of pred values of 100 DT\n ",np.var(DT_pred))

Variance of pred values of 100 DT
  12.211044000000005


In [25]:
pruned_DT_pred=[]                                    #Empty list for pruned DT predicted values    
for i in range(100):
    X_sple = X_train.sample(200,random_state=i)      #200 sample size
    y_sple = y_train.sample(200,random_state=i) 
    X_train1, X_valid, y_train1, y_valid = train_test_split(X_sple, y_sple, test_size = 0.20,random_state=0)  
    path = DT.cost_complexity_pruning_path(X_train1, y_train1)
    ccp_alphas = path.ccp_alphas
    
    mae_train = {}                                   #set to store alpha values
    mae_valid = {}
    for j in ccp_alphas:                             #optimum alphas using mini mae
        dt = DecisionTreeRegressor(ccp_alpha=j)
        dt.fit(X_train1,y_train1)
        pred_val = dt.predict(X_valid)               #pred on valid data
        pred_train = dt.predict(X_train1) 

        mae_train[i] = mean_absolute_error(y_train1,pred_train)       
        mae_valid[i] = mean_absolute_error(y_valid,pred_val)
    opt_ccp_alpha = min(mae_valid,key = mae_valid.get)        #Optimum Cost Complexity Parameter
        
    
    dt_prun = DecisionTreeRegressor(ccp_alpha=opt_ccp_alpha)
    dt_prun.fit(X_sple,y_sple)                         #Fitted pruned DT model using optimal cost
    pruned_DT_pred.append(dt.predict([test_obs])[0])

In [26]:
print("Variance of predicted values of 100 pruned DT\n",np.var(pruned_DT_pred))

Variance of predicted values of 100 pruned DT
 9.398585175264738


In [29]:
#random forest model
RF_DT_pred=[]
DT = RandomForestRegressor(n_estimators = 50)

for i in range(100):
    X_smpl = X_train.sample(200,random_state=i)   #random sample of size 200
    y_smpl = y_train.sample(200,random_state=i) 
    DT.fit(X_smpl,y_smpl)                         #Fitted fully growned decision tree model

    RF_DT_pred.append(DT.predict([test_obs])[0])

In [30]:
print("Variance of predicted values of 50 Random forest\n",np.var(RF_DT_pred))

Variance of predicted values of 50 Random forest
 1.1312506804000042
